In [1]:
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
import time
import csv
from datetime import datetime
import os

from bs4 import BeautifulSoup
from requests import get
import pandas as pd
from lxml import etree

from io import TextIOWrapper
from hdfs.client import Client

In [2]:
def clickToNext(nextButtonXpath, browser):
    element = browser.find_element_by_xpath(nextButtonXpath)
    actions = ActionChains(browser)
    actions.move_to_element(element).click().perform()
    time.sleep(2)

In [3]:
def clickToReview(reviewButtonXpath, browser):
    reviewButton = browser.find_element_by_xpath(reviewButtonXpath)
    reviewButton.location_once_scrolled_into_view
    time.sleep(0.5)
    reviewButton.click()
    time.sleep(0.5)

In [4]:
def webScrapVerizon(browser, maxPage, webPageHTMLText, reviewMap, config):
    ## click to review page
    clickToReview(config["reviewButtonXpath"], browser)
    
    page = 1
    while(page<=maxPage):
        ## click to next page
        clickToNext(config["nextButtonXpath"], browser)

        for title in browser.find_elements_by_xpath(config['title_xpath']):
            reviewMap["Title"].append(title.text)
            reviewMap["Device"].append("Samsung Galaxy S7")

        for user_info in browser.find_elements_by_xpath(config['user_info_xpath']):
            username, submissionDate = user_info.text.split(' - ')
            reviewMap["UserNickname"].append(username)
            reviewMap["SubmissionTime"].append(submissionDate)

        for review in browser.find_elements_by_xpath(config['review_xpath']):
            reviewMap["ReviewText"].append(review.text)

        print("---------------page finish: " + str(page))
        page = page+1
        
    
    browser.quit()
        
    return reviewMap

In [5]:
reviewMap = {"Device": [], "Title": [], 
             "ReviewText": [], "SubmissionTime": [],
            "UserNickname": []}

# reviewMap = pd.DataFrame(data = reviewMap)

In [6]:
config = {
    "verizonUrl":'https://www.verizonwireless.com/smartphones/samsung-galaxy-s7/',
    "chromePath":'D:/Verizon/chromedriver.exe',

    "user_info_xpath":'/html/body/div[4]/div/div[5]/div[1]/div/div[1]/div/div[1]/div/div[1]/div[6]/div[1]/div/div[*]/div[1]/p/div[1]/p',
    "review_xpath":'/html/body/div[4]/div/div[5]/div[1]/div/div[1]/div/div[1]/div/div[1]/div[6]/div[1]/div/div[*]/div[1]/p/div[2]/p',
    "title_xpath":'/html/body/div[4]/div/div[5]/div[1]/div/div[1]/div/div[1]/div/div[1]/div[6]/div[1]/div/div[*]/div[1]/p/p',

    "reviewButtonXpath":'/html/body/div[4]/div/div[5]/div[1]/div/div[1]/div/div[1]/div/div[1]/div[6]/div[1]/div/div[1]/div/span/span/div/div/div/ul/li[3]/button',
    "nextButtonXpath":'/html/body/div[4]/div/div[5]/div[1]/div/div[1]/div/div[1]/div/div[1]/div[6]/div[1]/div/div[17]/nav/ul/li[6]/a'
}

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
result = get(config["verizonUrl"], headers=headers)
webPageHTMLText = BeautifulSoup(result.content, "html.parser")

In [8]:
browser = webdriver.Chrome(executable_path='D:/Verizon/chromedriver.exe')
browser.get(config["verizonUrl"])

In [9]:
reviewMap2 = webScrapVerizon(browser, 17, webPageHTMLText, reviewMap, config)

---------------page finish: 1
---------------page finish: 2
---------------page finish: 3
---------------page finish: 4
---------------page finish: 5
---------------page finish: 6
---------------page finish: 7
---------------page finish: 8
---------------page finish: 9
---------------page finish: 10
---------------page finish: 11
---------------page finish: 12
---------------page finish: 13
---------------page finish: 14
---------------page finish: 15
---------------page finish: 16
---------------page finish: 17


In [10]:
reviewDF = pd.DataFrame(data = reviewMap2)

In [ ]:
def loadToHDFS(output_path, hdfsConnect):
    with hdfsConnect.write(os.path.join(output_path, "verizon_review.csv"), 
              encoding='utf-8') as writer:
    reviewDF.to_csv(writer)

In [97]:
# reviewDF.to_csv("verizon_review.csv")

In [17]:
fs = Client("http://0.0.0.0:19000/")

In [14]:
output_path = "D:\Hadoop\data\dfs\data_330"

In [18]:
with fs.write(os.path.join(output_path, "verizon_review.csv"), 
              encoding='utf-8') as writer:
    reviewDF.to_csv(writer)

ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=19000): Max retries exceeded with url: /webhdfs/v1/?op=GETHOMEDIRECTORY (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000021D5CA78AC8>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))